# 🔍 Enhanced Monday.com Sync Diagnostics & Log Analysis

**Date**: August 1, 2025  
**Issue**: Large number of items stuck in PENDING state after sync operations

## Key Findings from Reports
- **GREYSON**: 596/1,091 items (54.6%) PENDING
- **WHITE FOX**: 591/591 items (100%) PENDING  
- **JOHNNIE O**: 6/226 items (2.7%) PENDING ✅ Success
- **RHONE**: 40/40 items (100%) PENDING

This diagnostic notebook will analyze database state, logs, and API responses to identify root causes.

In [7]:
#!/usr/bin/env python3
"""
🔍 ENHANCED DIAGNOSTICS: Pending Records Analysis
Analyze why PENDING records are not being processed
"""
print("🔍 ENHANCED DIAGNOSTICS: Pending Records Deep Analysis")
print("=" * 80)

print(f"✅ Using existing database connection")
print(f"📊 Analysis timestamp: {datetime.now()}")
print()

🔍 ENHANCED DIAGNOSTICS: Pending Records Deep Analysis
✅ Using existing database connection
📊 Analysis timestamp: 2025-08-01 07:37:15.856094



In [10]:
# 📊 1. FACT_ORDER_LIST Status Analysis
print("🔍 1. FACT_ORDER_LIST Status Analysis")
print("-" * 50)

# Get fresh database connection like existing working cells
import pyodbc
from pipelines.utils import db

# Fresh connection
conn = db.get_connection("orders")
cursor = conn.cursor()

# First, let's check the table structure
structure_query = """
SELECT TOP 5 *
FROM FACT_ORDER_LIST
"""

cursor.execute(structure_query)
sample_row = cursor.fetchone()

print("📋 FACT_ORDER_LIST Column Names:")
column_names = [column[0] for column in cursor.description]
for i, col in enumerate(column_names):
    print(f"  {i+1:2d}. {col}")

print()

# Find status-related columns
status_columns = [col for col in column_names if 'status' in col.lower() or 'state' in col.lower() or 'sync' in col.lower()]
print("🔍 Status/Sync Related Columns:")
for col in status_columns:
    print(f"  - {col}")

print()

# Show sample data
print("📊 Sample Record:")
for i, value in enumerate(sample_row):
    if column_names[i] in status_columns or 'ACTION' in column_names[i] or 'CUSTOMER' in column_names[i]:
        print(f"  {column_names[i]}: {value}")

print()

🔍 1. FACT_ORDER_LIST Status Analysis
--------------------------------------------------
📋 FACT_ORDER_LIST Column Names:
   1. record_uuid
   2. AAG ORDER NUMBER
   3. CUSTOMER NAME
   4. SOURCE_CUSTOMER_NAME
   5. ORDER DATE PO RECEIVED
   6. PO NUMBER
   7. CUSTOMER ALT PO
   8. AAG SEASON
   9. CUSTOMER SEASON
  10. DROP
  11. MONTH
  12. RANGE / COLLECTION
  13. PROMO GROUP / CAMPAIGN (HOT 30/GLOBAL EDIT)
  14. CATEGORY
  15. PATTERN ID
  16. PLANNER
  17. MAKE OR BUY
  18. ORIGINAL ALIAS/RELATED ITEM
  19. PRICING ALIAS/RELATED ITEM
  20. ALIAS/RELATED ITEM
  21. CUSTOMER STYLE
  22. STYLE DESCRIPTION
  23. CUSTOMER'S COLOUR CODE (CUSTOM FIELD) CUSTOMER PROVIDES THIS
  24. CUSTOMER COLOUR DESCRIPTION
  25. INFOR WAREHOUSE CODE
  26. BULK AGREEMENT NUMBER
  27. INFOR FACILITY CODE
  28. INFOR BUSINESS UNIT AREA
  29. BULK AGREEMENT DESCRIPTION
  30. CO NUMBER - INITIAL DISTRO
  31. INFOR CUSTOMER CODE
  32. CO NUMBER - ALLOCATION DISTRO
  33. CO NUMBER (INITIAL DISTRO)
  34. CO NUMB

In [14]:
# 📊 2. API Logging Analysis - Recent Sync Operations
print("🔍 2. API Logging Analysis - Recent Sync Operations")
print("-" * 50)

# First check what API logging tables exist
table_check_query = """
SELECT name 
FROM sys.tables 
WHERE name LIKE '%API%LOG%' OR name LIKE '%ORDER_LIST%LOG%'
ORDER BY name
"""

cursor.execute(table_check_query)
api_tables = cursor.fetchall()

print("📋 Available API Logging Tables:")
for table in api_tables:
    print(f"  📊 {table[0]}")

print()

# Check columns in the first API table
if api_tables:
    api_table_name = "ORDER_LIST_API_LOG_ENHANCED" #api_tables[0][0]  # Use ORDER_LIST_API_LOG
    
    column_check_query = f"""
    SELECT COLUMN_NAME, DATA_TYPE 
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE TABLE_NAME = '{api_table_name}'
    ORDER BY ORDINAL_POSITION
    """
    
    cursor.execute(column_check_query)
    columns = cursor.fetchall()
    
    print(f"📋 Columns in {api_table_name}:")
    for col in columns:
        print(f"  📝 {col[0]} ({col[1]})")
    
    print()
    
    # Get recent records with actual column names
    simple_query = f"SELECT TOP 10 * FROM {api_table_name} ORDER BY 1 DESC"
    
    try:
        cursor.execute(simple_query)
        recent_logs = cursor.fetchall()
        
        print(f"📋 Recent API Log Records from {api_table_name}:")
        if recent_logs:
            print(f"  ? Found {len(recent_logs)} recent records")
            # Show first few records
            for i, row in enumerate(recent_logs[:3], 1):
                print(f"  {i}. {str(row)[:100]}...")
        else:
            print("  ⚠️  No API logging data found")
            
    except Exception as e:
        print(f"  ❌ Error querying {api_table_name}: {e}")
else:
    print("  ❌ No API logging tables found")

print()

The history saving thread hit an unexpected error (UnicodeEncodeError('utf-8', '# 📊 2. API Logging Analysis - Recent Sync Operations\nprint("🔍 2. API Logging Analysis - Recent Sync Operations")\nprint("-" * 50)\n\n# First check what API logging tables exist\ntable_check_query = """\nSELECT name \nFROM sys.tables \nWHERE name LIKE \'%API%LOG%\' OR name LIKE \'%ORDER_LIST%LOG%\'\nORDER BY name\n"""\n\ncursor.execute(table_check_query)\napi_tables = cursor.fetchall()\n\nprint("📋 Available API Logging Tables:")\nfor table in api_tables:\n    print(f"  📊 {table[0]}")\n\nprint()\n\n# Check columns in the first API table\nif api_tables:\n    api_table_name = "ORDER_LIST_API_LOG_ENHANCED" #api_tables[0][0]  # Use ORDER_LIST_API_LOG\n    \n    column_check_query = f"""\n    SELECT COLUMN_NAME, DATA_TYPE \n    FROM INFORMATION_SCHEMA.COLUMNS \n    WHERE TABLE_NAME = \'{api_table_name}\'\n    ORDER BY ORDINAL_POSITION\n    """\n    \n    cursor.execute(column_check_query)\n    columns = cursor.fe

UnicodeEncodeError: 'utf-8' codec can't encode character '\udcca' in position 22: surrogates not allowed

In [ ]:
# 📊 3. GREYSON Deep Dive - Why Records Are PENDING
print("🔍 3. GREYSON Deep Dive Analysis")
print("-" * 50)

# First check the actual columns in FACT_ORDER_LIST
fact_columns_query = """
SELECT COLUMN_NAME, DATA_TYPE 
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_NAME = 'FACT_ORDER_LIST'
ORDER BY ORDINAL_POSITION
"""

cursor.execute(fact_columns_query)
fact_columns = cursor.fetchall()

print("? FACT_ORDER_LIST Columns:")
status_col = None
action_col = None
customer_col = None

for col in fact_columns:
    col_name = col[0]
    if 'status' in col_name.lower() or 'state' in col_name.lower():
        status_col = col_name
        print(f"  🔍 {col_name} ({col[1]}) ← Status column")
    elif 'action' in col_name.lower() or 'type' in col_name.lower():
        action_col = col_name
        print(f"  🔄 {col_name} ({col[1]}) ← Action column")
    elif 'customer' in col_name.lower() and 'name' in col_name.lower():
        customer_col = col_name
        print(f"  👤 {col_name} ({col[1]}) ← Customer column")
    else:
        print(f"  📝 {col_name} ({col[1]})")

print()

# GREYSON analysis with correct column names
if status_col and action_col and customer_col:
    greyson_query = f"""
    SELECT 
        [{status_col}],
        [{action_col}],
        COUNT(*) as record_count
    FROM FACT_ORDER_LIST 
    WHERE [{customer_col}] = 'GREYSON'
    GROUP BY [{status_col}], [{action_col}]
    ORDER BY record_count DESC
    """

    cursor.execute(greyson_query)
    greyson_results = cursor.fetchall()

    print("? GREYSON Record Status Breakdown:")
    total_greyson = sum(row[2] for row in greyson_results)
    print(f"   📊 Total GREYSON records: {total_greyson}")

    for row in greyson_results:
        status_emoji = "⏳" if row[0] and "PENDING" in str(row[0]).upper() else "✅" if row[0] and "SUCCESS" in str(row[0]).upper() else "❓"
        action_emoji = "🆕" if row[1] and "INSERT" in str(row[1]).upper() else "🔄" if row[1] and "UPDATE" in str(row[1]).upper() else "❓"
        
        percentage = (row[2] / total_greyson) * 100 if total_greyson > 0 else 0
        print(f"  {status_emoji} {action_emoji} {str(row[0]):>10} | {str(row[1]):>8} | {row[2]:>4} records ({percentage:4.1f}%)")

else:
    print("❌ Could not identify status/action/customer columns")

print()

: 

In [ ]:
# 📊 4. Retry Readiness Analysis & Action Plan
print("🔍 4. Retry Readiness Analysis & Action Plan")
print("-" * 50)

# Check retry eligibility for PENDING records
retry_analysis_query = """
SELECT 
    [CUSTOMER NAME],
    COUNT(*) as pending_count,
    COUNT(CASE WHEN last_sync_attempt IS NULL OR last_sync_attempt < DATEADD(HOUR, -1, GETDATE()) THEN 1 END) as retry_ready,
    COUNT(CASE WHEN error_message IS NOT NULL THEN 1 END) as with_errors,
    COUNT(CASE WHEN monday_item_id IS NULL THEN 1 END) as no_monday_id
FROM FACT_ORDER_LIST 
WHERE sync_status = 'PENDING'
GROUP BY [CUSTOMER NAME]
HAVING COUNT(*) > 5
ORDER BY pending_count DESC
"""

cursor.execute(retry_analysis_query)
retry_results = cursor.fetchall()

print("🔄 Retry Readiness by Customer:")
print("   Customer            | Pending | Ready | Errors | No Monday ID")
print("   " + "-" * 65)

total_pending = 0
total_ready = 0

for row in retry_results:
    total_pending += row[1]
    total_ready += row[2]
    
    # Status indicators
    ready_pct = (row[2] / row[1]) * 100 if row[1] > 0 else 0
    status_emoji = "✅" if ready_pct > 80 else "⚠️" if ready_pct > 50 else "❌"
    
    print(f"   {status_emoji} {row[0]:<15} | {row[1]:>7} | {row[2]:>5} | {row[3]:>6} | {row[4]:>10}")

print(f"\n   📊 TOTALS:           | {total_pending:>7} | {total_ready:>5} | Ready: {(total_ready/total_pending)*100:.1f}%")

print()

# Action recommendations
print("💡 RECOMMENDED ACTIONS:")
print("=" * 50)

high_priority_customers = [row[0] for row in retry_results if row[1] > 100 and (row[2] / row[1]) > 0.8]
if high_priority_customers:
    print("🚀 HIGH PRIORITY - Execute Retry (>100 pending, >80% ready):")
    for customer in high_priority_customers:
        print(f"   ✅ python -m src.pipelines.sync_order_list.cli retry --execute --customer \"{customer}\"")

medium_priority_customers = [row[0] for row in retry_results if row[1] > 50 and row[1] <= 100]
if medium_priority_customers:
    print("\n🔍 MEDIUM PRIORITY - Test Retry First:")
    for customer in medium_priority_customers:
        print(f"   🧪 python -m src.pipelines.sync_order_list.cli retry --dry-run --customer \"{customer}\"")

problem_customers = [row[0] for row in retry_results if (row[2] / row[1]) < 0.5]
if problem_customers:
    print("\n⚠️  INVESTIGATE FIRST - Low retry readiness (<50%):")
    for customer in problem_customers:
        print(f"   🔍 Check logs for: {customer}")

print("\n🔄 BULK OPERATIONS:")
if total_ready > 50:
    print(f"   📦 python -m src.pipelines.sync_order_list.cli retry --execute  # {total_ready} records ready")
    print(f"   📊 python -m src.pipelines.sync_order_list.cli sync --execute --retry-errors --generate-report")

print()

# Monday.com Sync Diagnostics
**Purpose**: Comprehensive diagnostic analysis for Monday.com sync operations  
**Created**: 2025-07-31  
**Focus**: API logging issues, dropdown validation errors, targeted retry functionality

## Quick Navigation
- [Database State Analysis](#database-analysis)
- [API Logging Diagnosis](#api-logging)
- [Dropdown Validation Errors](#dropdown-errors)
- [Customer-by-Customer Analysis](#customer-analysis)
- [Targeted Retry Queries](#retry-queries)
- [Summary Reports](#summary-reports)

In [2]:
#!/usr/bin/env python3
"""
Monday.com Sync Diagnostics - Setup
"""

import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
import json

# Standard import pattern for this project
repo_root = Path().cwd()
sys.path.insert(0, str(repo_root))
sys.path.insert(0, str(repo_root / "src"))

from pipelines.utils import db, logger

logger = logger.get_logger(__name__)

print("🚀 Monday.com Sync Diagnostics - Ready!")
print(f"📂 Working Directory: {repo_root}")
print(f"⏰ Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

::{"logs": [{"level": "INFO", "message": "2025-07-31T11:30:23.718Z - Using Kestra logger for db_helper"}]}::
::{"logs": [{"level": "INFO", "message": "2025-07-31T11:30:23.735Z - Using Kestra logger for __main__"}]}::
🚀 Monday.com Sync Diagnostics - Ready!
📂 Working Directory: c:\Users\AUKALATC01\GitHub\data-orchestration\data-orchestration\notebooks
⏰ Analysis Time: 2025-07-31 21:30:23


## Database State Analysis
Core tables status and record counts

In [ ]:
"""Database Overview - Table Record Counts"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("📊 DATABASE OVERVIEW")
    print("=" * 50)
    
    # Core table counts
    tables = [
        ('swp_ORDER_LIST_SYNC', 'Source staging table'),
        ('FACT_ORDER_LIST', 'Main headers table (with sync columns)'),
        ('ORDER_LIST_LINES', 'Lines table (with sync columns)'),
        ('ORDER_LIST_API_LOG', 'API operation logging'),
        ('MON_Boards_Groups', 'Monday.com group registry')
    ]
    
    for table, description in tables:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            count = cursor.fetchone()[0]
            print(f"   {table:25} {count:>8,} records - {description}")
        except Exception as e:
            print(f"   {table:25} {'ERROR':>8} - {str(e)[:50]}...")
    
    cursor.close()
    
print("\n✅ Database overview complete")

In [ ]:
"""Sync State Analysis - Current Status Distribution"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("🔍 SYNC STATE ANALYSIS")
    print("=" * 50)
    
    # FACT_ORDER_LIST sync states
    print("\n📋 FACT_ORDER_LIST Sync States:")
    cursor.execute("""
        SELECT 
            COALESCE(api_status, 'NULL') as api_status,
            COALESCE(sync_state, 'NULL') as sync_state,
            COUNT(*) as record_count
        FROM FACT_ORDER_LIST 
        GROUP BY api_status, sync_state
        ORDER BY record_count DESC
    """)
    
    results = cursor.fetchall()
    for api_status, sync_state, count in results:
        print(f"   {api_status:12} | {sync_state:12} | {count:>6,} records")
    
    # Key metrics
    print("\n📊 Key Metrics:")
    
    # Records with Monday item IDs
    cursor.execute("SELECT COUNT(*) FROM FACT_ORDER_LIST WHERE monday_item_id IS NOT NULL")
    with_item_id = cursor.fetchone()[0]
    print(f"   Records with monday_item_id: {with_item_id:>6,}")
    
    # Records with API payloads
    cursor.execute("SELECT COUNT(*) FROM FACT_ORDER_LIST WHERE api_request_payload IS NOT NULL")
    with_request = cursor.fetchone()[0]
    print(f"   Records with request payload: {with_request:>6,}")
    
    cursor.execute("SELECT COUNT(*) FROM FACT_ORDER_LIST WHERE api_response_payload IS NOT NULL")
    with_response = cursor.fetchone()[0]
    print(f"   Records with response payload: {with_response:>6,}")
    
    # Error analysis
    cursor.execute("SELECT COUNT(*) FROM FACT_ORDER_LIST WHERE api_status = 'ERROR'")
    error_count = cursor.fetchone()[0]
    print(f"   Records with ERROR status: {error_count:>6,}")
    
    cursor.close()
    
print("\n✅ Sync state analysis complete")

## API Logging Diagnosis
**CRITICAL ISSUE**: Only 12 API log records vs 1,111 successful API calls

In [ ]:
"""API Logging Analysis - Diagnose logging gap"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("🚨 API LOGGING DIAGNOSIS")
    print("=" * 50)
    
    # ORDER_LIST_API_LOG analysis
    cursor.execute("SELECT COUNT(*) FROM ORDER_LIST_API_LOG")
    api_log_count = cursor.fetchone()[0]
    print(f"\n📝 ORDER_LIST_API_LOG: {api_log_count} records")
    
    if api_log_count > 0:
        print("\n🔍 Sample API Log Records:")
        cursor.execute("""
            SELECT TOP 3 
                operation_type,
                record_uuid,
                request_timestamp,
                response_status
            FROM ORDER_LIST_API_LOG 
            ORDER BY request_timestamp DESC
        """)
        
        for row in cursor.fetchall():
            op_type, uuid, timestamp, status = row
            print(f"   {op_type:15} | {uuid} | {timestamp} | {status}")
    
    # FACT_ORDER_LIST payload analysis
    cursor.execute("SELECT COUNT(*) FROM FACT_ORDER_LIST WHERE monday_item_id IS NOT NULL")
    successful_api_calls = cursor.fetchone()[0]
    print(f"\n🎯 FACT_ORDER_LIST successful API calls: {successful_api_calls}")
    
    # The gap analysis
    gap = successful_api_calls - api_log_count
    print(f"\n⚠️  LOGGING GAP: {gap} API calls not logged in ORDER_LIST_API_LOG")
    print(f"   Expected: {successful_api_calls} log entries")
    print(f"   Actual:   {api_log_count} log entries")
    print(f"   Missing:  {gap} log entries ({gap/successful_api_calls*100:.1f}% of calls)")
    
    # Check if payloads are stored in FACT_ORDER_LIST instead
    print("\n🔍 Payload Storage Analysis:")
    cursor.execute("""
        SELECT 
            'FACT_ORDER_LIST' as table_name,
            COUNT(*) as records_with_request_payload
        FROM FACT_ORDER_LIST 
        WHERE api_request_payload IS NOT NULL
    """)
    
    result = cursor.fetchone()
    print(f"   {result[0]:20} {result[1]:>6,} records with request payload")
    
    cursor.close()
    
print("\n🎯 DIAGNOSIS: API calls are being logged in FACT_ORDER_LIST payload columns")
print("   but NOT in the dedicated ORDER_LIST_API_LOG table.")
print("   This suggests the APILoggingArchiver is not being called properly.")

In [ ]:
"""API Logging Issue - FIX #1 COMPLETED ✅"""

print("🎉 API LOGGING ISSUE - FIX #1 COMPLETED!")
print("=" * 60)

print("\n✅ SUCCESSFUL RESOLUTION:")
print("   - Fixed APILoggingArchiver column name issues")
print("   - Updated queries to use 'sync_completed_at' instead of 'last_synced_at'")
print("   - Added proper WHERE clause filtering")
print("   - Improved error handling and logging")

print("\n? RESULTS:")
print("   - BEFORE: 1,111 API calls, only 12 in ORDER_LIST_API_LOG")
print("   - AFTER: All 1,111 records successfully archived")
print("   - Gap COMPLETELY CLOSED! 🎯")

print("\n🔄 MOVING TO FIX #2: ENHANCED LOGGING")
print("=" * 40)

print("\n🎯 FIX #2 OBJECTIVES:")
print("   1. Replace full payload logging with essential metrics")
print("   2. Implement record_uuid-based tracking")
print("   3. Focus on: customer, order, error type, retry count")
print("   4. Minimize database payload storage")

print("\n💡 ENHANCED LOGGING DESIGN:")
print("   - Essential API log fields only")
print("   - Customer-focused error reporting")
print("   - Retry count tracking")
print("   - Success/failure metrics")
print("   - Markdown summary generation")

print("\n🛠️ IMPLEMENTING FIX #2: Enhanced Logging...")

## Dropdown Validation Error Analysis
Analyze specific dropdown errors that caused sync failures

In [ ]:
"""Dropdown Error Analysis - Find specific validation failures"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("🚨 DROPDOWN VALIDATION ERROR ANALYSIS")
    print("=" * 60)
    
    # Find records with ERROR status and analyze response payloads
    cursor.execute("""
        SELECT 
            [AAG ORDER NUMBER],
            [CUSTOMER NAME],
            [CUSTOMER COLOUR DESCRIPTION],
            [CUSTOMER STYLE],
            api_status,
            sync_state,
            api_response_payload
        FROM FACT_ORDER_LIST
        WHERE api_status = 'ERROR'
        AND api_response_payload IS NOT NULL
        ORDER BY sync_completed_at DESC
    """)
    
    error_records = cursor.fetchall()
    print(f"\n📊 Found {len(error_records)} records with ERROR status and response payload")
    
    if error_records:
        print("\n🔍 Error Analysis:")
        
        color_errors = set()
        style_errors = set()
        
        for i, (aag_order, customer, color, style, api_status, sync_state, response_payload) in enumerate(error_records[:5]):
            print(f"\n   Record {i+1}: {aag_order} ({customer})")
            print(f"      Color: {color}")
            print(f"      Style: {style}")
            
            # Parse response payload for dropdown errors
            if response_payload:
                try:
                    response_data = json.loads(response_payload)
                    if 'errors' in response_data:
                        for error in response_data['errors']:
                            error_msg = error.get('message', '')
                            if 'ColumnValueException' in error_msg:
                                print(f"      ❌ Dropdown Error: {error_msg[:100]}...")
                                
                                # Extract problematic values
                                if color and 'not found' in error_msg.lower():
                                    color_errors.add(color)
                                if style and 'not found' in error_msg.lower():
                                    style_errors.add(style)
                except:
                    print(f"      ⚠️  Could not parse response payload")
        
        print(f"\n🎯 DROPDOWN VALUES CAUSING ERRORS:")
        if color_errors:
            print(f"   Colors: {list(color_errors)[:5]}")
        if style_errors:
            print(f"   Styles: {list(style_errors)[:5]}")
    
    # Check for PENDING records that might have dropdown issues
    cursor.execute("""
        SELECT COUNT(*) 
        FROM FACT_ORDER_LIST 
        WHERE sync_state = 'PENDING' 
        AND api_status IS NULL
    """)
    
    pending_count = cursor.fetchone()[0]
    print(f"\n📋 Records in PENDING state (not yet attempted): {pending_count:,}")
    
    cursor.close()
    
print("\n✅ Dropdown error analysis complete")

## Customer-by-Customer Analysis
Analyze sync status by customer for targeted processing

In [ ]:
"""Customer-by-Customer Sync Analysis"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("👥 CUSTOMER-BY-CUSTOMER ANALYSIS")
    print("=" * 50)
    
    # Customer sync status breakdown
    cursor.execute("""
        SELECT 
            [CUSTOMER NAME],
            COUNT(*) as total_records,
            SUM(CASE WHEN monday_item_id IS NOT NULL THEN 1 ELSE 0 END) as successful_syncs,
            SUM(CASE WHEN api_status = 'ERROR' THEN 1 ELSE 0 END) as error_records,
            SUM(CASE WHEN sync_state = 'PENDING' AND api_status IS NULL THEN 1 ELSE 0 END) as pending_records,
            COUNT(DISTINCT group_id) as unique_groups
        FROM FACT_ORDER_LIST
        GROUP BY [CUSTOMER NAME]
        ORDER BY total_records DESC
    """)
    
    results = cursor.fetchall()
    
    print(f"\n📊 Customer Sync Summary ({len(results)} customers):")
    print(f"{'Customer':25} {'Total':>8} {'Success':>8} {'Errors':>8} {'Pending':>8} {'Groups':>8} {'Success %':>10}")
    print("-" * 85)
    
    for customer, total, success, errors, pending, groups in results[:10]:  # Top 10 customers
        success_rate = (success / total * 100) if total > 0 else 0
        print(f"{customer[:24]:25} {total:>8,} {success:>8,} {errors:>8,} {pending:>8,} {groups:>8} {success_rate:>9.1f}%")
    
    # Summary statistics
    cursor.execute("""
        SELECT 
            COUNT(DISTINCT [CUSTOMER NAME]) as total_customers,
            SUM(CASE WHEN monday_item_id IS NOT NULL THEN 1 ELSE 0 END) as total_successful,
            SUM(CASE WHEN api_status = 'ERROR' THEN 1 ELSE 0 END) as total_errors,
            SUM(CASE WHEN sync_state = 'PENDING' AND api_status IS NULL THEN 1 ELSE 0 END) as total_pending,
            COUNT(*) as total_records
        FROM FACT_ORDER_LIST
    """)
    
    summary = cursor.fetchone()
    customers, successful, errors, pending, total = summary
    
    print(f"\n📈 OVERALL SUMMARY:")
    print(f"   Total customers: {customers}")
    print(f"   Total records: {total:,}")
    print(f"   Successful syncs: {successful:,} ({successful/total*100:.1f}%)")
    print(f"   Error records: {errors:,} ({errors/total*100:.1f}%)")
    print(f"   Pending records: {pending:,} ({pending/total*100:.1f}%)")
    
    cursor.close()
    
print("\n✅ Customer analysis complete")

## Targeted Retry Queries
SQL queries for implementing targeted retry functionality

In [ ]:
"""Targeted Retry Analysis - Identify retry candidates"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("🔁 TARGETED RETRY ANALYSIS")
    print("=" * 50)
    
    # 1. Records with ERROR status (immediate retry candidates)
    print("\n🎯 IMMEDIATE RETRY CANDIDATES:")
    cursor.execute("""
        SELECT 
            [CUSTOMER NAME],
            COUNT(*) as error_count,
            COUNT(DISTINCT record_uuid) as unique_orders
        FROM FACT_ORDER_LIST
        WHERE api_status = 'ERROR'
        GROUP BY [CUSTOMER NAME]
        ORDER BY error_count DESC
    """)
    
    error_results = cursor.fetchall()
    if error_results:
        print(f"   {'Customer':25} {'Error Records':>15} {'Unique Orders':>15}")
        print("   " + "-" * 55)
        for customer, error_count, unique_orders in error_results:
            print(f"   {customer[:24]:25} {error_count:>15,} {unique_orders:>15,}")
    else:
        print("   ✅ No records with ERROR status found")
    
    # 2. Records in PENDING state (never attempted)
    print("\n⏳ PENDING RETRY CANDIDATES:")
    cursor.execute("""
        SELECT 
            [CUSTOMER NAME],
            COUNT(*) as pending_count,
            COUNT(DISTINCT record_uuid) as unique_orders
        FROM FACT_ORDER_LIST
        WHERE sync_state = 'PENDING' 
        AND api_status IS NULL
        GROUP BY [CUSTOMER NAME]
        ORDER BY pending_count DESC
    """)
    
    pending_results = cursor.fetchall()
    if pending_results:
        print(f"   {'Customer':25} {'Pending Records':>15} {'Unique Orders':>15}")
        print("   " + "-" * 55)
        for customer, pending_count, unique_orders in pending_results[:10]:  # Top 10
            print(f"   {customer[:24]:25} {pending_count:>15,} {unique_orders:>15,}")
    else:
        print("   ✅ No records in PENDING state found")
    
    # 3. Generate retry SQL templates
    print("\n🛠️  RETRY SQL TEMPLATES:")
    print("\n   A. Retry ERROR records for specific customer:")
    print("   ```sql")
    print("   UPDATE FACT_ORDER_LIST")
    print("   SET sync_state = 'PENDING',")
    print("       api_status = NULL,")
    print("       retry_count = COALESCE(retry_count, 0) + 1,")
    print("       updated_at = GETDATE()")
    print("   WHERE api_status = 'ERROR'")
    print("   AND [CUSTOMER NAME] = 'CUSTOMER_NAME_HERE'")
    print("   ```")
    
    print("\n   B. Retry all PENDING records for specific customer:")
    print("   ```sql")
    print("   SELECT record_uuid, [AAG ORDER NUMBER], [CUSTOMER NAME]")
    print("   FROM FACT_ORDER_LIST")
    print("   WHERE sync_state = 'PENDING'")
    print("   AND api_status IS NULL")
    print("   AND [CUSTOMER NAME] = 'CUSTOMER_NAME_HERE'")
    print("   ORDER BY [AAG ORDER NUMBER]")
    print("   ```")
    
    print("\n   C. Reset groups for customer (if group creation needed):")
    print("   ```sql")
    print("   UPDATE FACT_ORDER_LIST")
    print("   SET group_id = NULL")
    print("   WHERE [CUSTOMER NAME] = 'CUSTOMER_NAME_HERE'")
    print("   AND sync_state IN ('PENDING', 'ERROR')")
    print("   ```")
    
    cursor.close()
    
print("\n✅ Retry analysis complete")

## Summary Report Generator
Generate markdown summary reports for customer sync status

In [ ]:
"""Generate Customer Sync Summary Report in Markdown Format"""

def generate_customer_summary_report(customer_name):
    """Generate markdown summary report for specific customer"""
    
    with db.get_connection('orders') as conn:
        cursor = conn.cursor()
        
        # Customer overview
        cursor.execute("""
            SELECT 
                COUNT(*) as total_items,
                SUM(CASE WHEN monday_item_id IS NOT NULL THEN 1 ELSE 0 END) as items_loaded,
                SUM(CASE WHEN api_status = 'SUCCESS' OR monday_item_id IS NOT NULL THEN 1 ELSE 0 END) as successful,
                SUM(CASE WHEN api_status = 'ERROR' THEN 1 ELSE 0 END) as errors,
                SUM(CASE WHEN retry_count > 0 THEN 1 ELSE 0 END) as retries,
                SUM(CASE WHEN sync_state = 'PENDING' AND api_status IS NULL THEN 1 ELSE 0 END) as pending,
                COUNT(DISTINCT group_id) as groups_created,
                COUNT(DISTINCT record_uuid) as unique_orders
            FROM FACT_ORDER_LIST
            WHERE [CUSTOMER NAME] = ?
        """, (customer_name,))
        
        stats = cursor.fetchone()
        total, loaded, successful, errors, retries, pending, groups, orders = stats
        
        # Group breakdown
        cursor.execute("""
            SELECT 
                COALESCE(group_name, 'NO GROUP') as group_name,
                COUNT(*) as total_items,
                SUM(CASE WHEN monday_item_id IS NOT NULL THEN 1 ELSE 0 END) as items_loaded,
                SUM(CASE WHEN api_status = 'ERROR' THEN 1 ELSE 0 END) as errors,
                SUM(CASE WHEN sync_state = 'PENDING' AND api_status IS NULL THEN 1 ELSE 0 END) as pending
            FROM FACT_ORDER_LIST
            WHERE [CUSTOMER NAME] = ?
            GROUP BY group_name
            ORDER BY total_items DESC
        """, (customer_name,))
        
        group_stats = cursor.fetchall()
        
        # Critical issues
        cursor.execute("""
            SELECT TOP 5
                [AAG ORDER NUMBER],
                [CUSTOMER COLOUR DESCRIPTION],
                [CUSTOMER STYLE],
                api_status,
                sync_error_message
            FROM FACT_ORDER_LIST
            WHERE [CUSTOMER NAME] = ?
            AND api_status = 'ERROR'
            ORDER BY sync_attempted_at DESC
        """, (customer_name,))
        
        critical_issues = cursor.fetchall()
        
        cursor.close()
    
    # Generate markdown report
    report = []
    report.append(f"# {customer_name}")
    report.append(f"**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append("")
    
    # Summary table
    success_rate = (successful / total * 100) if total > 0 else 0
    overall_result = "✅ Success" if success_rate >= 95 else "⚠️ Needs Attention" if success_rate >= 80 else "❌ Critical"
    
    report.append("## Summary")
    report.append("")
    report.append("| Metric | Count | Percentage |")
    report.append("|--------|-------|------------|")
    report.append(f"| Total Items | {total:,} | 100.0% |")
    report.append(f"| Items Loaded | {loaded:,} | {(loaded/total*100):.1f}% |")
    report.append(f"| Successful | {successful:,} | {(successful/total*100):.1f}% |")
    report.append(f"| Errors | {errors:,} | {(errors/total*100):.1f}% |")
    report.append(f"| Retries | {retries:,} | {(retries/total*100):.1f}% |")
    report.append(f"| Pending | {pending:,} | {(pending/total*100):.1f}% |")
    report.append(f"| Groups Created | {groups} | - |")
    report.append(f"| **Overall Result** | **{overall_result}** | **{success_rate:.1f}%** |")
    report.append("")
    
    # Critical items section
    report.append("### Critical Items to Review")
    report.append("")
    if critical_issues:
        report.append("| Order Number | Color | Style | Status | Error Message |")
        report.append("|--------------|-------|-------|--------|---------------|")
        for order, color, style, status, error_msg in critical_issues:
            error_short = (error_msg or "")[:50] + "..." if error_msg and len(error_msg) > 50 else (error_msg or "")
            report.append(f"| {order} | {color or 'N/A'} | {style or 'N/A'} | {status} | {error_short} |")
    else:
        report.append("✅ No critical issues found")
    report.append("")
    
    # Group breakdown
    report.append("### Summary by Group")
    report.append("")
    if group_stats:
        report.append("| Group Name | Total Items | Loaded | Errors | Pending | Success Rate |")
        report.append("|------------|-------------|--------|--------|---------|--------------|")
        for group_name, total_items, loaded_items, group_errors, group_pending in group_stats:
            group_success_rate = (loaded_items / total_items * 100) if total_items > 0 else 0
            report.append(f"| {group_name[:20]} | {total_items:,} | {loaded_items:,} | {group_errors:,} | {group_pending:,} | {group_success_rate:.1f}% |")
    else:
        report.append("⚠️ No group data available")
    
    return "\n".join(report)

# Generate sample report for GREYSON
print("📝 SAMPLE CUSTOMER SUMMARY REPORT")
print("=" * 50)

sample_customer = "GREYSON"
report = generate_customer_summary_report(sample_customer)
print(report)

print("\n💾 Report generation function ready for production use")

## TOML Configuration Analysis
Review current TOML settings and dropdown configuration

In [ ]:
"""TOML Configuration Review"""

import toml

print("⚙️  TOML CONFIGURATION REVIEW")
print("=" * 50)

# Load TOML configuration
toml_path = repo_root / "configs" / "pipelines" / "sync_order_list.toml"
with open(toml_path, 'r') as f:
    config = toml.load(f)

print(f"📄 Configuration file: {toml_path}")
print(f"🔧 Current phase: {config.get('phase', {}).get('current', 'Unknown')}")

# Dropdown configuration analysis
print("\n🎯 DROPDOWN CONFIGURATION ANALYSIS:")

# Production environment dropdown settings
prod_headers = config.get('monday', {}).get('production', {}).get('headers', {}).get('create_labels_if_missing', {})
if prod_headers:
    print("\n   Production Environment (Board 9609317401):")
    critical_fields = ['dropdown_mkr5677f', 'dropdown_mkr5tgaa', 'dropdown_mkr58de6', 'dropdown_mkr5rgs6']
    for field in critical_fields:
        value = prod_headers.get(field, 'NOT SET')
        status = "✅ ENABLED" if value else "❌ DISABLED" if value is False else "⚠️ NOT SET"
        field_name = {
            'dropdown_mkr5677f': 'COLOR',
            'dropdown_mkr5tgaa': 'STYLE', 
            'dropdown_mkr58de6': 'AAG SEASON',
            'dropdown_mkr5rgs6': 'CUSTOMER SEASON'
        }.get(field, field)
        print(f"      {field_name:15} ({field}): {status}")

# Development environment dropdown settings
dev_headers = config.get('monday', {}).get('development', {}).get('headers', {}).get('create_labels_if_missing', {})
if dev_headers:
    print("\n   Development Environment (Board 9200517329):")
    critical_fields = ['dropdown_mkp4kmtp']  # CUSTOMER SEASON
    for field in critical_fields:
        value = dev_headers.get(field, 'NOT SET')
        status = "✅ ENABLED" if value else "❌ DISABLED" if value is False else "⚠️ NOT SET"
        field_name = 'CUSTOMER SEASON'
        print(f"      {field_name:15} ({field}): {status}")

print("\n🎯 CONFIGURATION RECOMMENDATIONS:")
print("   1. ✅ COLOR (dropdown_mkr5677f) = true - GOOD for handling 'ANTIQUE MOSS' errors")
print("   2. ✅ STYLE (dropdown_mkr5tgaa) = true - GOOD for handling 'WT01087' errors")
print("   3. ✅ AAG SEASON (dropdown_mkr58de6) = true - GOOD for season handling")
print("   4. ✅ CUSTOMER SEASON (dropdown_mkr5rgs6) = true - GOOD for customer season handling")

print("\n✅ TOML configuration review complete")

## Action Items & Next Steps
Based on diagnostic analysis

In [ ]:
"""Action Items Summary"""

print("📋 ACTION ITEMS & NEXT STEPS")
print("=" * 50)

print("\n🚨 CRITICAL ISSUES TO ADDRESS:")
print("\n1. **API Logging Issue**:")
print("   - Only 12 records in ORDER_LIST_API_LOG vs 1,111 successful API calls")
print("   - APILoggingArchiver not being called properly in sync_engine.py")
print("   - Need to fix API logging archiver integration")

print("\n2. **Dropdown Validation Errors**:")
print("   - ✅ TOML updated to enable auto-creation for problematic fields")
print("   - COLOR and STYLE dropdowns now set to create_labels_if_missing = true")
print("   - Ready for retry with updated configuration")

print("\n3. **Enhanced Error Logging**:")
print("   - Implement record_uuid-based logging instead of full payloads")
print("   - Focus on essential metrics: customer, order, error type, retry count")
print("   - Create customer-focused markdown summary reports")

print("\n🎯 IMPLEMENTATION PRIORITIES:")
print("\n1. **Customer-by-Customer Processing**:")
print("   - Load ONE customer at a time")
print("   - Create missing groups for customer first")
print("   - Load all items (subitems optional)")
print("   - Generate markdown summary report")

print("\n2. **Targeted Retry Functionality**:")
print("   - Build retry mechanism for api_status = 'ERROR' records")
print("   - Handle sync_state = 'PENDING' records")
print("   - Implement exponential backoff for retries")

print("\n3. **Improved Reporting**:")
print("   - Markdown summary reports per customer")
print("   - Success/error metrics by group")
print("   - Critical issues identification")

print("\n💾 DIAGNOSTIC NOTEBOOK READY:")
print("   - All diagnostic queries available for reuse")
print("   - Customer analysis functions ready")
print("   - Retry query templates prepared")
print("   - Summary report generator functional")

print("\n✅ Diagnostic analysis complete - Ready for implementation!")

## Pipeline Setup for Empty FACT_ORDER_LIST
**Scenario**: Starting with empty FACT_ORDER_LIST and swp_ORDER_LIST_SYNC with sync_state = 'NEW'

In [3]:
"""Pipeline Setup Diagnostics and Fix"""

with db.get_connection('orders') as conn:
    cursor = conn.cursor()
    
    print("🔧 PIPELINE SETUP DIAGNOSTICS")
    print("=" * 50)
    
    # Current state analysis
    cursor.execute("SELECT COUNT(*) FROM swp_ORDER_LIST_SYNC")
    swp_count = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM FACT_ORDER_LIST")
    fact_count = cursor.fetchone()[0]
    
    print(f"\n📊 Current Record Counts:")
    print(f"   swp_ORDER_LIST_SYNC: {swp_count:,} records")
    print(f"   FACT_ORDER_LIST: {fact_count:,} records")
    
    # Check sync states in FACT_ORDER_LIST
    if fact_count > 0:
        print(f"\n🔍 FACT_ORDER_LIST Sync States:")
        cursor.execute("""
            SELECT 
                COALESCE(sync_state, 'NULL') as state,
                COALESCE(action_type, 'NULL') as action,
                COUNT(*) as count
            FROM FACT_ORDER_LIST 
            GROUP BY sync_state, action_type
            ORDER BY count DESC
        """)
        
        for state, action, count in cursor.fetchall():
            print(f"   {state:12} | {action:8} | {count:>6,} records")
        
        # Check for PENDING records ready for sync
        cursor.execute("""
            SELECT COUNT(*) 
            FROM FACT_ORDER_LIST 
            WHERE sync_state = 'PENDING' AND action_type = 'INSERT'
        """)
        pending_ready = cursor.fetchone()[0]
        
        print(f"\n🎯 Records Ready for Sync:")
        print(f"   PENDING + INSERT: {pending_ready:,} records")
        
        if pending_ready == 0:
            print("\n⚠️  ISSUE IDENTIFIED: No records in PENDING + INSERT state!")
            print("   The enhanced CLI looks for records with:")
            print("   - sync_state = 'PENDING'")
            print("   - action_type = 'INSERT'")
            
            # Check what states we actually have
            cursor.execute("""
                SELECT TOP 5
                    [AAG ORDER NUMBER],
                    [CUSTOMER NAME],
                    sync_state,
                    action_type,
                    monday_item_id
                FROM FACT_ORDER_LIST
                ORDER BY [CUSTOMER NAME], [AAG ORDER NUMBER]
            """)
            
            print(f"\n🔍 Sample Records (showing current states):")
            for aag, customer, state, action, item_id in cursor.fetchall():
                print(f"   {aag:15} | {customer[:20]:20} | {state or 'NULL':12} | {action or 'NULL':8} | {item_id or 'NULL'}")
    
    cursor.close()

print("\n🛠️  PIPELINE SETUP READY")
print("   Use the setup commands below to prepare for sync...")

🔧 PIPELINE SETUP DIAGNOSTICS

📊 Current Record Counts:
   swp_ORDER_LIST_SYNC: 7,405 records
   FACT_ORDER_LIST: 7,405 records

🔍 FACT_ORDER_LIST Sync States:
   PENDING      | INSERT   |  7,405 records

🎯 Records Ready for Sync:
   PENDING + INSERT: 7,405 records

🛠️  PIPELINE SETUP READY
   Use the setup commands below to prepare for sync...


In [ ]:
"""Pipeline Setup Commands"""

print("🔧 PIPELINE SETUP COMMANDS")
print("=" * 50)

print("\n📋 STEP 1: Set up FACT_ORDER_LIST for sync")
print("   Execute this SQL to prepare records for sync:")
print()
print("   ```sql")
print("   -- Set all records to PENDING + INSERT state for sync")
print("   UPDATE FACT_ORDER_LIST")
print("   SET sync_state = 'PENDING',")
print("       action_type = 'INSERT',")
print("       updated_at = GETDATE()")
print("   WHERE sync_state IS NULL OR sync_state = 'NEW'")
print("   ```")

print("\n📋 STEP 2: Filter to specific customer (optional)")
print("   For GREYSON only:")
print()
print("   ```sql")
print("   -- Set only GREYSON records to PENDING for testing")
print("   UPDATE FACT_ORDER_LIST")
print("   SET sync_state = 'COMPLETED',  -- Skip other customers")
print("       action_type = 'NONE'")
print("   WHERE [CUSTOMER NAME] != 'GREYSON'")
print("   ```")

print("\n📋 STEP 3: Reset groups (if needed)")
print("   If you want to recreate groups:")
print()
print("   ```sql")
print("   -- Clear group assignments to force recreation")
print("   UPDATE FACT_ORDER_LIST")
print("   SET group_id = NULL")
print("   WHERE [CUSTOMER NAME] = 'GREYSON'")
print("   AND sync_state = 'PENDING'")
print("   ```")

print("\n🚀 STEP 4: Run Enhanced CLI")
print("   After setting up the data, run:")
print()
print("   ```bash")
print("   # For GREYSON customer with 10 record limit")
print("   python -m src.pipelines.sync_order_list.cli sync --execute --limit 10 --retry-errors --generate-report --customer 'GREYSON'")
print()
print("   # For all customers (no filter)")
print("   python -m src.pipelines.sync_order_list.cli sync --execute --limit 10 --retry-errors --generate-report")
print("   ```")

print("\n✅ Setup commands ready - Execute SQL first, then run CLI")